In [1]:
# Pacotes iniciais
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['figure.figsize']=(10,10)

import os
os.chdir('..')
os.getcwd()

'/media/fellipegm/Data/Drive/07 Projetos/Projeto-CESAR'

### Load data, clean and perform the PCA

In [2]:
from functions.data_loading import DataLoad

data = DataLoad()

### Test of the most simple model (Linear Regression), using the PCA variables

In [3]:
from sklearn.linear_model import LinearRegression
from functions.metrics import Metrics

linreg = dict()
y_pred_linreg = dict()
stats_linreg = dict()

variables = ['Tmax', 'Tmin']

for variable in variables:
    linreg[variable] = dict()
    y_pred_linreg[variable] = dict()
    stats_linreg[variable] = dict()
    for i in range(1, 26):
        linreg[variable][i] = LinearRegression()
        linreg[variable][i].fit(data.df_train[i], data.y_train[i][f'Next_{variable}'])
        y_pred_linreg[variable][i] = linreg[variable][i].predict(data.df_test[i])
        interm = Metrics(data.y_test[i][f'Next_{variable}'], y_pred_linreg[variable][i])
        stats_linreg[variable][i] = interm.fit()

### Check the same scores for the LDAPS Tmax and Tmin variables

The same scores are calculated for the LDAPS model Tmax and Tmin air temperatures. Need to check if something is improving.

In [9]:
stats_LDAPS = dict()

for variable in variables:
    stats_LDAPS[variable] = dict()
    for i in range(1, 26):
        interm = Metrics(data.y_test[i][f'Next_{variable}'], data.df_test[i][f'LDAPS_{variable}_lapse'])
        stats_LDAPS[variable][i] = interm.fit()

### Comparison of the residual mean and R^2

In [12]:
# This code is ugly af and can be improved greatly using dict or list. Too tired and with running out of time to improve it now.
# I feel that even the csv comparison file could be much better.

header = 'Index'
for i in range(1, 26): header += f',Station {i}'
header += '\n'

for variable in variables:
    mean = 'residual mean Linear Regression'
    mean_LDAPS = 'residual mean LDAPS'
    r2 = 'R2 Linear Regression'
    r2_LDAPS = 'R2 LDAPS'
    improv_r2 = 'improvement R2'
    for i in range(1, 26):
        mean += ',{:.2f}'.format(stats_linreg[variable][i]['residual_mean'])
        mean_LDAPS += ',{:.2f}'.format(stats_LDAPS[variable][i]['residual_mean'])
        r2 += ',{:.2f}'.format(stats_linreg[variable][i]['r2'])
        r2_LDAPS += ',{:.2f}'.format(stats_LDAPS[variable][i]['r2'])
        improv_r2 += ',{:.2f}'.format((stats_linreg[variable][i]['r2'] - stats_LDAPS[variable][i]['r2']) / stats_LDAPS[variable][i]['r2'])
    
    mean += '\n'
    mean_LDAPS += '\n'
    r2 += '\n'
    r2_LDAPS += '\n'
    improv_r2 += '\n'

    with open(f'./output/linreg_comparison_{variable}.csv', 'w') as outfile:
        outfile.write(header + mean + mean_LDAPS + r2 + r2_LDAPS + improv_r2)




In [ ]:
stats = ['r2', 'residual_mean']

for stat in stats:
    for variable in variables:
        plt.figure()
        plt.title(f'{variable} - {stat}')
        r2 = [stats_linreg[variable][i][stat] for i in range(1, 26)]
        r2_LDAPS = [stats_LDAPS[variable][i][stat] for i in range(1, 26)]
            